In [1]:
import os
import json
import sys

root_dir = os.path.join(os.getcwd(), "..")
sys.path.append(root_dir)

In [2]:
import torch.nn.init as init
import torch.nn as nn
import torch
from torch.distributions import MultivariateNormal

from scipy.linalg import eigvals
import numpy as np

In [3]:
def transform_matrix(A):
    # Making positive definite
    eigenvalues = eigvals(A)
    min_eigenvalue = min(eigenvalues)
    if min_eigenvalue <= 0 + 1e-4:
        A = A + (abs(min_eigenvalue) + 1e-4)*np.eye(A.shape[0])

    # Normalizing by dividing by the trace
    trace_A = np.trace(A)
    if trace_A > 0:
        A = A / trace_A

    return A 

In [4]:
def nfm_init(config, model):
    """
    Initialize the weight of convolution layers with nfm as covariance matrix.
    ONLY FOR VGG11
    """
    conv_layer_index = 0
    for name, module in model.features.named_children():
        if isinstance(module, nn.Conv2d):
            # load the nfm
            nfm_path = os.path.join(config['model']['nfm_path'], f"layer_{conv_layer_index}.csv")
            nfm = np.loadtxt(nfm_path, delimiter=',')
            nfm = transform_matrix(nfm)
            nfm = torch.from_numpy(nfm).float()
            init_conv_with_cov(module, nfm)
            conv_layer_index += 1

    return model

In [5]:
# load in the nfm
for i in range(7):
    print(i)
    nfm_path = os.path.join('/Users/keyu/Documents/GitHub/DSC180/nfm', f"layer_{i}.csv")
    nfm = np.loadtxt(nfm_path, delimiter=',')
    
    # test if we can create a multivariate gaus distribution with this nfm
    means = torch.zeros(nfm.shape[0])
    nfm = torch.from_numpy(nfm).float()
    # m = MultivariateNormal(means, nfm)
    
    min_eigenvalue = min(eigvals(nfm))
    nfm_new = nfm + (abs(min_eigenvalue) + 1e-4)*np.eye(nfm.shape[0])
    nfm_new = nfm_new / np.trace(nfm_new)
    min(eigvals(nfm_new))
    m = MultivariateNormal(means, nfm_new)

0
1


ValueError: Expected parameter covariance_matrix (Tensor of shape (576, 576)) of distribution MultivariateNormal(loc: torch.Size([576]), covariance_matrix: torch.Size([576, 576])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[ 1.8021e-03, -5.4840e-05,  1.4849e-05,  ..., -7.8554e-05,
         -1.7664e-05,  1.5991e-05],
        [-5.4630e-05,  1.9953e-03,  3.3843e-05,  ..., -4.0491e-05,
          1.2779e-05,  2.2799e-05],
        [ 1.4779e-05,  3.4274e-05,  1.6210e-03,  ...,  7.0209e-06,
          1.5944e-05,  2.5231e-05],
        ...,
        [-7.8527e-05, -4.0540e-05,  7.0689e-06,  ...,  1.4906e-03,
          1.2985e-04,  5.0007e-05],
        [-1.7680e-05,  1.2842e-05,  1.5958e-05,  ...,  1.2987e-04,
          1.4837e-03,  1.7284e-04],
        [ 1.5992e-05,  2.2764e-05,  2.5207e-05,  ...,  5.0000e-05,
          1.7289e-04,  1.5295e-03]], dtype=torch.float64)

In [9]:
nfm_path = os.path.join('/Users/keyu/Documents/GitHub/DSC180/nfm', f"layer_{1}.csv")
nfm = np.loadtxt(nfm_path, delimiter=',')

# test if we can create a multivariate gaus distribution with this nfm
means = torch.zeros(nfm.shape[0])
nfm = torch.from_numpy(nfm).float()
# m = MultivariateNormal(means, nfm)

min_eigenvalue = min(eigvals(nfm))
print(min_eigenvalue)
nfm_new = nfm + (abs(min_eigenvalue) + 1e-5)*np.eye(nfm.shape[0])

nfm_new = nfm_new / np.trace(nfm_new)
print(min(eigvals(nfm_new)))
m = MultivariateNormal(means, nfm_new)

(-0.42696163+0j)
(2.8605191544833842e-05+0j)


ValueError: Expected parameter covariance_matrix (Tensor of shape (576, 576)) of distribution MultivariateNormal(loc: torch.Size([576]), covariance_matrix: torch.Size([576, 576])) to satisfy the constraint PositiveDefinite(), but found invalid values:
tensor([[ 1.8011e-03, -5.3946e-05,  1.4607e-05,  ..., -7.7272e-05,
         -1.7376e-05,  1.5731e-05],
        [-5.3738e-05,  1.9910e-03,  3.3291e-05,  ..., -3.9830e-05,
          1.2571e-05,  2.2427e-05],
        [ 1.4538e-05,  3.3715e-05,  1.6228e-03,  ...,  6.9064e-06,
          1.5684e-05,  2.4820e-05],
        ...,
        [-7.7246e-05, -3.9879e-05,  6.9536e-06,  ...,  1.4946e-03,
          1.2774e-04,  4.9191e-05],
        [-1.7392e-05,  1.2632e-05,  1.5698e-05,  ...,  1.2775e-04,
          1.4878e-03,  1.7002e-04],
        [ 1.5731e-05,  2.2392e-05,  2.4795e-05,  ...,  4.9185e-05,
          1.7006e-04,  1.5329e-03]], dtype=torch.float64)

In [21]:
# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(nfm)

# Modify eigenvalues to be positive
positive_eigenvalues = np.abs(eigenvalues) + 1e-5  # Adding a small value to ensure positivity

# Reconstruct the positive definite matrix
nfm_positive_definite = eigenvectors @ np.diag(positive_eigenvalues) @ eigenvectors.T
nfm_positive_definite = torch.from_numpy(nfm_positive_definite).float()

In [15]:
nfm_new = nfm + (abs(min_eigenvalue) + 1e-5)*np.eye(nfm.shape[0])

In [16]:
min(eigvals(nfm_new))

(9.457264069088346e-06+0j)

In [22]:
MultivariateNormal(means, nfm_positive_definite)

MultivariateNormal(loc: torch.Size([576]), covariance_matrix: torch.Size([576, 576]))

(2.0168894709188777e-09+0j)